In [271]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers


dataset_path = keras.utils.get_file("SPY.csv", "http://localhost:8000/data/daily/SPY.csv")

column_names = ['Date','Open','High','Low','Close','Adj Close','Volume'] 
raw_dataset = pd.read_csv(dataset_path, 
#                           names=column_names, 
                          dtype={'Close': np.float64,'Open': np.float64,'High': np.float64,'Adj Close': np.float64, 'Volume': np.float64},
                          header=0,
                          na_values = "?", 
                          comment='\t',
                          sep=",",
                          skipinitialspace=True)


dataset = raw_dataset.copy()
dataset = dataset.sort_values(by=['Date'],ascending=False)

print len(dataset)

dataset.head(5)

dataset_stats = dataset.describe()
dataset_stats = dataset_stats.transpose()
NUM_INPUT_NEURONS = 64
NUM_OUTPUT_NEURONS = 1
# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old


def convert_labels_to_category(labels): 
    # Simplification - If positive return, 1, else 0
    # return map(lambda arr: 1 if arr[0] > 1 else 0, labels)
    # rounding simpliciation, 10th of percentage
    return map(lambda arr: map(lambda val: round(val,2),arr), labels)

def convert_to_train(raw_dataset):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(5, len(dataset) - NUM_INPUT_NEURONS):

        feature_dataset = dataset[i:i+NUM_INPUT_NEURONS].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
            #dataset['Close'].iloc[i-3] / latest_close, # 3 day trade
            #dataset['Close'].iloc[i-5] / latest_close, # 5 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]
converted_feature_set = convert_to_train(dataset)
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init)
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(converted_feature_set[0][0])]),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dense(16, activation=tf.nn.relu),
    layers.Dense(NUM_OUTPUT_NEURONS)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer='sgd',
                metrics=[
                    'mae',
                #    'accuracy'
                ])
  return model

tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

model = build_model()
model.summary()



6445
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_71 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_72 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_73 (Dense)             (None, 1)                 17        
Total params: 6,785
Trainable params: 6,785
Non-trainable params: 0
_________________________________________________________________


In [272]:

#len(converted_feature_set[0][0])
train_data = np.array(converted_feature_set[0][900:])
train_labels = np.array(converted_feature_set[1][900:])

test_data = np.array(converted_feature_set[0][:900])
test_labels = np.array(converted_feature_set[1][:900])


In [273]:
print train_data
print train_labels

[[ 0.         -0.01299094 -0.02653142 ...  0.01014289  0.00864392
   0.01988605]
 [ 0.         -0.01336683 -0.01099929 ...  0.02135741  0.03245537
   0.03112367]
 [ 0.          0.00233632 -0.00316383 ...  0.04521778  0.04390365
   0.05120469]
 ...
 [ 0.          0.00280309  0.00630582 ... -0.00840926 -0.00911115
  -0.00490652]
 [ 0.          0.00351258  0.00983838 ... -0.01194773 -0.00773128
   0.00281097]
 [ 0.          0.0063481   0.00775853 ... -0.0112835  -0.00070409
   0.00141044]]
[[1.01]
 [0.99]
 [0.99]
 ...
 [1.  ]
 [1.  ]
 [1.  ]]


In [274]:

history = model.fit(
    train_data, train_labels,
    epochs=40, validation_split = 0.2, verbose=1,
    callbacks=[tensorboard]
)

Train on 4380 samples, validate on 1096 samples
Epoch 1/40
4380/4380 [==============================] - 0s 107us/step - loss: 0.0902 - mean_absolute_error: 0.1708 - val_loss: 4.5877e-04 - val_mean_absolute_error: 0.0168
Epoch 2/40
4380/4380 [==============================] - 0s 46us/step - loss: 7.7786e-04 - mean_absolute_error: 0.0205 - val_loss: 2.5693e-04 - val_mean_absolute_error: 0.0124
Epoch 3/40
4380/4380 [==============================] - 0s 47us/step - loss: 5.7626e-04 - mean_absolute_error: 0.0170 - val_loss: 2.0496e-04 - val_mean_absolute_error: 0.0109
Epoch 4/40
4380/4380 [==============================] - 0s 46us/step - loss: 4.9419e-04 - mean_absolute_error: 0.0154 - val_loss: 1.6583e-04 - val_mean_absolute_error: 0.0099
Epoch 5/40
4380/4380 [==============================] - 0s 47us/step - loss: 4.4988e-04 - mean_absolute_error: 0.0145 - val_loss: 1.4900e-04 - val_mean_absolute_error: 0.0094
Epoch 6/40
4380/4380 [==============================] - 0s 47us/step - loss: 4.2

In [275]:
outputs = model.predict(test_data)


In [276]:

# one day return
actual = map(lambda arr: arr[0],outputs)
# signal step for our policy network
signals = map(lambda arr: sum(arr) / NUM_OUTPUT_NEURONS,outputs)
# primitive policy temporarily in place of a RL policy network
trades = map(lambda signal: 1 if round(signal,2) > 1 else 0,signals)

In [277]:
df = pd.DataFrame({
    'signal':signals,
#    'actual':test_labels,
    'actual': actual,
    'trade':trades,
})

In [278]:
df['entry_success'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 1) else 0,axis=1)
df['entry_failure'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 1) else 0,axis=1)
df['avoid_success'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 0) else 0,axis=1)
df['avoid_failure'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 0) else 0,axis=1)


In [282]:
# primitive policy - replace with a policy network which maximizes reward
def label_success (row):
    return 0 if (row['entry_failure'] == 1) else 1

success = df.apply (lambda row: label_success (row),axis=1)
df['success'] = success;
df

,actual,signal,trade,entry_success,entry_failure,avoid_success,avoid_failure,success
0,1.003408,1.003408,0,0,0,0,1,1
1,0.999825,0.999825,0,0,0,1,0,1
2,1.002791,1.002791,0,0,0,0,1,1
3,1.004313,1.004313,0,0,0,0,1,1
4,1.002149,1.002149,0,0,0,0,1,1
5,1.000294,1.000294,0,0,0,0,1,1
6,1.002940,1.002940,0,0,0,0,1,1
7,1.002093,1.002093,0,0,0,0,1,1
8,1.007495,1.007495,1,1,0,0,0,1
9,1.002644,1.002644,0,0,0,0,1,1


In [283]:
print '\nLoss trades'
print sum(df['entry_failure'])
print sum(df['entry_failure']) / 900.0

print '\nNon-loss trades'
print sum(df['success'])
print sum(df['success']) / 900.0

print '\nWin trades'
print sum(df['entry_success'])
print sum(df['entry_success']) / 900.0


Loss trades
0
0.0

Non-loss trades
900
1.0

Win trades
134
0.148888888889
